In [ ]:
import httpx
import pandas as pd
import os 
import json

from src.download.patients import PatientsDownload
from src.download.abwasser_downloader import SewagedataDownloader
from src.utils.graph_configurations import get_sewagedata_timeline_plot

# Info: 
Dieses Skript arbeitet mit den Abwasserdaten (Viruslast im Wasser) des RKIs. Es werden folgende Use Cases abgedeckt: 

Für einzelne Patient:innen:
1) Für einzelne Patient:innen Abwasserdaten anschauen
2) Filterung der Abwasserdaten (nach Virusart)
3) Filterung der Abwasserdaten (nach Normalisierung)
4) Weiterverarbeitung der Daten: Mean/ Median in einem Zeitraum berechnen 
5) Fehlende start_date und end_date bei einem/r einzelnen Patient:in -> Berechnung des Zeitraums anhand des Diagnose-Datums

Für mehrere Patient:innen:
6) Für mehrere Patient:innen Abwasserdaten anschauen
7) Fehlende start_date und end_date bei mehreren Patient:in -> Berechnung des Zeitraums anhand des Diagnose-Datums

### 1) Use Case: Von einer Datei alle Patient:innen extrahieren und für einzelne Patient:innen Abwasserdaten anschauen

In [34]:
# INPUT from the User Interface 
START_DATE = input("Bitte geben Sie den Startzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar: ")
END_DATE = input("Bitte geben Sie den Endzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar: ")

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_small.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)
interesting_patient = patients.get_patient_by_id("595")

In [ ]:
# Get Abwasser Daten for Patient
sewage_data = SewagedataDownloader()
data = sewage_data.get_sewage_data_patient(start_date = START_DATE, end_date= END_DATE, patient = interesting_patient)

In [ ]:
data.head()

### 2) Use Case: Additional filtering - Filterung nach Typ

In [ ]:
START_DATE = input("Bitte geben Sie den Startzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar:  ")
END_DATE = input("Bitte geben Sie den Endzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar:  ")
VIRUS_TYPE = input("Bitte geben Sie den Virus-Typ ein. Optionen sind: SARS-CoV-2, Influenza A, Influenza B oder Influenza A+B:")

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_small.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)
interesting_patient = patients.get_patient_by_id("595")

In [ ]:
# Get Abwasser Daten for Patient
sewage_data = SewagedataDownloader()
data = sewage_data.get_sewage_data_patient(start_date = START_DATE, end_date= END_DATE, patient = interesting_patient, virus_type = VIRUS_TYPE)

In [ ]:
# data without missings
data_ = data.dropna(subset=['viruslast']) # Drop rows with NaN values in the column
get_sewagedata_timeline_plot(data = data_, column = 'viruslast')

### 3) Use Case: Additional filtering - Filterung nach Normalisierung

In [ ]:
START_DATE = input("Bitte geben Sie den Startzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar:  ")
END_DATE = input("Bitte geben Sie den Endzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar: ")
IS_NORMALIZED = input("Bitte geben Sie an ob die Werte normalisiert sein sollen. Optionen: ja oder nein")

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_small.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)
interesting_patient = patients.get_patient_by_id("595")

In [ ]:
# Get Abwasser Daten for Patient
sewage_data = SewagedataDownloader()
data = sewage_data.get_sewage_data_patient(start_date = START_DATE, end_date= END_DATE, patient = interesting_patient, is_normalsierung = IS_NORMALIZED)

### 4) Use Case: Daten noch weiter verarbeiten (Mean / Median)

In [ ]:
START_DATE = input("Bitte geben Sie den Startzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar:  ")
END_DATE = input("Bitte geben Sie den Endzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar:  ")
VIRUS_TYPE = input("Bitte geben Sie den Virus-Typ ein. Optionen sind: SARS-CoV-2, Influenza A, Influenza B oder Influenza A+B:")

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_small.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)
interesting_patient = patients.get_patient_by_id("595")

In [ ]:
# Get Abwasser Daten for Patient
sewage_data = SewagedataDownloader()
data = sewage_data.get_sewage_data_patient(start_date = START_DATE, end_date= END_DATE, patient = interesting_patient, virus_type = VIRUS_TYPE)

In [ ]:
# apply mean to the data
print(f"Durchschnittlich gemessene Viruslast (mean) in Genkopien pro Liter für den Zeitraum {START_DATE} bis {END_DATE} für den Virus-Typ {VIRUS_TYPE}: {round(data['viruslast'].mean())} (gerundet)")
print(f"Durchschnittlich vorhergesagte Viruslast (mean) in Genkopien pro Liter für den Zeitraum {START_DATE} bis {END_DATE} für den Virus-Typ {VIRUS_TYPE}: {round(data['loess_vorhersage'].mean())} (gerundet)")

In [ ]:
# apply median to the data
print(f"Durchschnittlich gemessene Viruslast (median) in Genkopien pro Liter für den Zeitraum {START_DATE} bis {END_DATE} für den Virus-Typ {VIRUS_TYPE}: {round(data['viruslast'].median())} (gerundet)")
print(f"Durchschnittlich vorhergesagte Viruslast (median) in Genkopien pro Liter für den Zeitraum {START_DATE} bis {END_DATE} für den Virus-Typ {VIRUS_TYPE}: {round(data['loess_vorhersage'].median())} (gerundet)")

### 5) Use Case: Von einer Datei alle Patient:innen extrahieren und für einzelne Patient:innen Abwasserdaten anschauen - start_date und end_date werden nicht angegeben und automatisch berechnet

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_small.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)
interesting_patient = patients.get_patient_by_id("595")

In [ ]:
# Get Abwasser Daten for Patient
sewage_data = SewagedataDownloader()
data = sewage_data.get_sewage_data_patient(patient = interesting_patient, start_date = None, end_date = None)

In [ ]:
print(f"Da kein Start- und Enddatum angegeben wurde, werden Daten 3 Monate vor- und nach des Diagnose-Datums bereitgestellt (Zeitraum: {data['datum'].min().strftime('%Y-%m-%d')} - {data['datum'].max().strftime('%Y-%m-%d')})")

### 6) Use Case: über mehrere Patient:innen loopen und einen Datensatz für alle

In [ ]:
# INPUT from the User Interface 
START_DATE = input("Bitte geben Sie den Startzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar:  ")
END_DATE = input("Bitte geben Sie den Endzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind nur Daten zwischen 01.06.2022 and 31.12.2024 verfügbar:  ")

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_large.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)

sewage_data = SewagedataDownloader()
return_data = sewage_data.get_sewage_data_patient_collection(patients= patients.patients, start_date = START_DATE, end_date = END_DATE)

In [ ]:
return_data.head()

In [ ]:
# simple table to show how many patient are in which standort
return_data_deduplicated = return_data.drop_duplicates(subset=['standort', 'patient_id'])
grouped = return_data_deduplicated.groupby('standort')['patient_id'].count().reset_index()
grouped.columns = ['standort', 'num_patients']

print(grouped)

### 7) Use Case: Über mehrere Patient:innen loopen ohne Start- oder Enddatum

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_large.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)

sewage_data = SewagedataDownloader()
return_data = sewage_data.get_sewage_data_patient_collection(patients= patients.patients, start_date = None, end_date = None)